## Sesión 4

En esta última sesión se propone realizar un ejercicio libre con el objetivo de mejorar lo máximo posible el acierto en la tarea MNIST. A modo de orientación se entiende por una buena tasa de acierto en esta tarea cuando obtenemos más de un 99% de acierto.

Para conseguir este resultado podréis probar pipelines que preprocesen los datos adecuadamente. Probar diferentes clasificadores e incluso combinación de los mismos.

In [1]:
## imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# ...
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



/home/joan/anaconda3/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
#data =mnist.data.astype(np.uni32).tonumpy()
data=np.float32(data)


/home/joan/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

In [3]:
# Partición de Datos

x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size = 0.2, random_state= 23)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state=23)


Ejercicio: probar todo lo necesario para obtener un acc > 99%

In [4]:
## Ejercicio obtener acc>99% 



for n in [2,4,6,8,16]:
    clf = make_pipeline(PCA(n_components=n), HistGradientBoostingClassifier(max_iter=1000, max_depth=10, max_bins=40), GradientBoostingClassifier())
    clf.fit(x_train,y_train)
    ypred = clf.predict(x_val)
    acc = accuracy_score(y_val, ypred)
    print("clasificador:", clf, "para n: ", n, "precision: ", acc)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'HistGradientBoostingClassifier(max_bins=40, max_depth=10, max_iter=1000)' (type <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'>) doesn't

In [5]:
from sklearn.ensemble import VotingClassifier


clf = HistGradientBoostingClassifier(min_samples_leaf = 5, max_bins= 32)
clf3 = make_pipeline(PCA(n_components=64), KNeighborsClassifier(n_neighbors=3))
clf4= KNeighborsClassifier(n_neighbors=5)
clf5 =make_pipeline(PCA(n_components=64), LinearDiscriminantAnalysis())
clf6 = HistGradientBoostingClassifier(min_samples_leaf=3)

clf7 = XGBClassifier(
            n_estimators = n,
            subsample = 0.75,
            #in_child_weight = 3,
            colsample_bytree = 0.70,
            random_state = 23,
            max_depth = 8,
        )

eclf = VotingClassifier(estimators=[('HGB', clf3), ('knn3', clf7)],
                        voting='soft')
eclf = eclf.fit(x_train, y_train)
ypred = eclf.predict(x_test)
acc = accuracy_score(y_test, ypred)
print(acc)

0.9791428571428571


In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

data_tr=np.concatenate((x_train,x_val),axis=0)
targets_tr=np.concatenate((y_train,y_val),axis=0)

#clf1 = HistGradientBoostingClassifier(min_samples_leaf=5, max_bins=32)
#clf2 = KNeighborsClassifier(n_neighbors=3)
clf3 = make_pipeline(PCA(n_components=64),KNeighborsClassifier(n_neighbors=3))
#clf4 = KNeighborsClassifier(n_neighbors=5)
#clf5 = make_pipeline(PCA(n_components=64),LinearDiscriminantAnalysis())
clf6 = HistGradientBoostingClassifier(min_samples_leaf=3)


eclf = VotingClassifier(
    estimators=[('HGB', clf3), ('knn3', clf6)],
    voting='soft')
eclf = eclf.fit(data_tr, targets_tr)
ypred = eclf.predict(x_test)
acc = accuracy_score(y_test, ypred)

print(acc)


AxisError: axis 1 is out of bounds for array of dimension 1

In [10]:
#n=70, precision = 0.7
from xgboost import XGBClassifier
for n in [50, 70, 90,100]:
    for samples in [0.7, 0.75,0.8,0.85]:
        clf = XGBClassifier(
            n_estimators = n,
            subsample = samples,
            #in_child_weight
            colsample_bytree = samples,
            random_state = 23,
            max_depth = 8,
        )
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("para n estimadores: ", n, "sub_samples: ", samples, "precision :", acc)


para n estimadores:  50 sub_samples:  0.7 precision : 0.9755714285714285
para n estimadores:  50 sub_samples:  0.75 precision : 0.976
para n estimadores:  50 sub_samples:  0.8 precision : 0.9748571428571429
para n estimadores:  50 sub_samples:  0.85 precision : 0.975
para n estimadores:  70 sub_samples:  0.7 precision : 0.9785714285714285
para n estimadores:  70 sub_samples:  0.75 precision : 0.9775714285714285
para n estimadores:  70 sub_samples:  0.8 precision : 0.9768571428571429
para n estimadores:  70 sub_samples:  0.85 precision : 0.9757142857142858
para n estimadores:  90 sub_samples:  0.7 precision : 0.9794285714285714
para n estimadores:  90 sub_samples:  0.75 precision : 0.9778571428571429
para n estimadores:  90 sub_samples:  0.8 precision : 0.9781428571428571


KeyboardInterrupt: 